In [1]:
from corpus import *
from vsm import *

Running from notebook, using serial load function.
[20, 40, 60, 80, 100]
/home/hongliang/inpho/handian/models/handian2-freq5-freq2-N23231312-LDA-K{0}-document-1000.npz


# 测试函数
link.value应该是从这个矩阵搞出来的
v.model.top_doc 是一个（20,18414）的矩阵
np.shape(v.model.top_doc[:][:,].sum(axis=1)) 是一个（20，）的矩阵,每一个元素都是2.54279e+06左右的数字,sum是69310878.375
sum(v.model.top_doc[:]) 是一个18414个元素的列表，每一个元素2000到10000，列表总和为69310878.375
 

sankey图中，左边的link的值，大于右边的总和,把不重要的置零。然后计算。
我们是，只计算大的

In [2]:
print topic_range[0]
k = topic_range[0]
v = lda_v[k]

csvpath = 'dynastyof book.csv'
jsonpath = 'sankey/multilayer.json'

20
Loading LDA data from /home/hongliang/inpho/handian/models/handian2-freq5-freq2-N23231312-LDA-K20-document-1000.npz


In [3]:
overall_topics = v.aggregate_doc_topics(v.labels, normed_sum=True)
topics = dict()
for a in overall_topics:
    topics[a[0]] = a[1]

In [4]:
# show characters rather than words


example = [[u'handian2\u300e\u53f2\u90e8\u300f\u7f16\u5e74\u7eed\u8d44\u6cbb\u901a\u9274\u957f\u7f16'],u'handian2\u300e\u96c6\u90e8\u300f\u522b\u96c6\u5e7d\u68a6\u5f71']
# example = handian
case_list_righ = str(example).replace('u\'','\'')
print case_list_righ.decode("unicode-escape")

def printlist(temp):
    example = temp
    case_list_righ = str(example).replace('u\'','\'')
    print case_list_righ.decode("unicode-escape")

[['handian2『史部』编年续资治通鉴长编'], 'handian2『集部』别集幽梦影']


To change the number of words printed per topic, use the `print_len` argument:

In [5]:
# folders = set([])
# for label in v.labels[:]:
#     folders.add(''.join(label.split('/')[:-1]))
# folders_dict = dict([])
# for i in folders:
#     folders_dict[i] = []

# count = 0
# for label in v.labels[:]:
#     if ''.join(label.split('/')[:-1]) in folders:
#         temp = ''.join(label.split('/')[:-1])
#         folders_dict[temp].append(count)
#     count += 1
# # printlist(folders_dict)
# # print count
# bigfolder = []
# folders_copy = folders.copy()
# for a in folders_copy:
#     if len(folders_dict[a]) <= 80:
#         folders.remove(a)
#         folders_dict.pop(a)
         
# print len(folders),len(folders_dict)
# # printlist(folders_dict)


# 经部48  史部 135 （大于8 有81） 子部334（大于16 有75） 集部127（大于16 有52）

In [6]:
folders = set([])
folders_dict = dict([])
for label in v.labels[:]:
    temp = ''.join(label.split('/')[:-1])
    if '『经部'.decode('utf-8') in temp:
        folders.add(temp)
        folders_dict[temp] = []

count = 0
for label in v.labels[:]:
    temp = ''.join(label.split('/')[:-1])
    if '『经部'.decode('utf-8') in temp:
     
        folders_dict[temp].append(count)
    count += 1

# folders_copy = folders.copy()
# for a in folders_copy:
#     if len(folders_dict[a]) <= 15:
#         folders.remove(a)
#         folders_dict.pop(a)
         
print len(folders),len(folders_dict)



48 48


In [7]:
# for label in v.labels[:]:
#     temp = ''.join(label.split('/')[:-1])
#     if '『子部'.decode('utf-8') in temp or '『经部'.decode('utf-8') in temp:
#         print temp


In [8]:
import csv
import json
    

dynasties_set = set([])
csvfile = open(csvpath,'r')
jsonfile = open('test.json','w')
filednames = ('book','dynasty')
reader = csv.DictReader(csvfile,filednames)
dynasties = [[row['dynasty'].decode('utf-8'),row['book'].decode('utf-8')] for row in reader][1:]
for row in dynasties: 
    searchword = 'handian2'+ ''.join(row[1].strip(',').split('/'))
    try:
        folders_dict[searchword]
        dynasties_set.add(row[0])
    except:
        'already delete'
printlist(dynasties_set)


set(['三国', '春秋战国', '春秋', '汉', '南宋', '西周', '北宋', '明', '秦汉', '唐', '西汉', '战国', '清', '东晋', '东汉'])


In [9]:
## depth 0
import topicexplorer.lib.color as color
topic_color = dict(get_topic_colors(v))

handian = dict()
handian['nodes']=[]

for a in v.aggregate_doc_topics(v.labels[:], normed_sum=True):
    temp = dict()
    temp['color'] = color.rgb2hex(topic_color[a[0]])
    temp['depth'] = 0
    temp['name'] = "T%s"%a[0]
#    temp['weight'] = a[1]
    temp['words'] = ', '.join(v.topics()[a[0]][:10]['word'])
    handian['nodes'].append(temp)
print 'handian has %s nodes'%len(handian['nodes'])
# depth 1

for a in dynasties_set:
    temp = dict()
    temp['depth'] = 1
    temp['name'] = a
    handian['nodes'].append(temp)
print 'handian has %s nodes'%len(handian['nodes'])
    
    
# depth 2 folders name  
for a in folders:
    temp = dict()
    temp['depth'] = 2
    temp['name'] = a
    handian['nodes'].append(temp)
print 'handian has %s nodes'%len(handian['nodes'])
    
# 问题，模型里面计算出来的文件夹只有644个，而csv中有八百多个。初步怀疑是空文件的问题。也可以以模型为准。
# print len(folders)
# print count1
# printlist(handian)

handian has 20 nodes
handian has 35 nodes
handian has 83 nodes


In [10]:
# handian['links']=[]



# csvfile = open(csvpath,'r')
# jsonfile = open('test.json','w')
# filednames = ('book','dynasty','year')
# reader = csv.DictReader(csvfile,filednames)
# dynasties = [[row['dynasty'].decode('utf-8'),row['book'].decode('utf-8')] for row in reader][1:]
# count2 = 0
# for a in dynasties:
#     # add 'handian2' so that can be the exact index to search
#     searchword = 'handian2'+ ''.join(a[1].strip(',').split('/'))
#     try:
#         index_number = folders_dict[searchword]
#         for topic in v.aggregate_doc_topics(v.labels[index_number[0]:index_number[-1]+1], normed_sum=True)[:1]:
#             temp = dict([])
#             temp['source'] = a[0]
#             temp['target'] = 'handian2'+''.join(a[1].strip(',').split('/'))
#             temp['topic'] = 'T%s'%topic[0]
#             temp['value'] = topic[1]*100
#             temp['color'] = color.rgb2hex(topic_color[topic[0]])
#             handian['links'].append(temp)

#     except:
#         count2 += 1
# print 'handian has %s links'%len(handian['links'])
# print count2

In [11]:
handian['links']=[]



csvfile = open(csvpath,'r')
jsonfile = open('test.json','w')
filednames = ('book','dynasty','year')
reader = csv.DictReader(csvfile,filednames)
dynasties = [[row['dynasty'].decode('utf-8'),row['book'].decode('utf-8')] for row in reader][1:]
count2 = 0
for a in dynasties:
    # add 'handian2' so that can be the exact index to search
    searchword = 'handian2'+ ''.join(a[1].strip(',').split('/'))
    try:
        index_number = folders_dict[searchword]
        for topic in v.aggregate_doc_topics(v.labels[index_number[0]:index_number[-1]+1], normed_sum=True):

            if topic[1] >= topics[topic[0]]**0.5:
                
                temp = dict([])
                temp['source'] = a[0]
                temp['target'] = 'handian2'+''.join(a[1].strip(',').split('/'))
                temp['topic'] = 'T%s'%topic[0]
                
                temp['value'] = sum(v.model.top_doc[topic[0]][index_number[0]:index_number[-1]+1])
#                 sum_topic_value = sum(v.model.top_doc[topic[0]][index_number[0]:index_number[-1]+1])
#                 temp['value'] = math.log(sum_topic_value,100)
                temp['color'] = color.rgb2hex(topic_color[topic[0]])
                handian['links'].append(temp)

    except:
        count2 += 1
print 'handian has %s links'%len(handian['links'])
print count2

handian has 64 links
779


In [12]:
handian_level2 = handian['links']
for topic in range(0,100):
    for dynasty in dynasties_set:
        temp = {'topic':'T%s'%topic,'source':'T%s'%topic,'target':dynasty,'value':0}
        for a in handian_level2:
            if a['topic']==temp['topic']:
                if a['source']==temp['target']:
                    temp['value'] = temp['value'] + a['value']
                    temp['color'] = color.rgb2hex(topic_color[topic])
        
        if temp['value'] != 0:
            handian['links'].append(temp)
print 'handian has %s links'%len(handian['links'])

handian has 106 links


In [13]:
# handian['links']=[]


# csvfile = open(csvpath,'r')
# jsonfile = open('test.json','w')
# filednames = ('book','dynasty','year')
# reader = csv.DictReader(csvfile,filednames)
# dynasties = [[row['dynasty'].decode('utf-8'),row['book'].decode('utf-8')] for row in reader][1:]
# count2 = 0
# for a in dynasties:
# # add 'handian2' so that can be the exact index to search
#     searchword = 'handian2'+ ''.join(a[1].strip(',').split('/'))
#     temp = dict([])
#     temp['source'] = a[0]
#     temp['target'] = '/'.join(a[1].strip(',').split('/'))

#     try:
#         folders_dict[searchword]
#         temp['value'] = len(folders_dict[searchword])
#         handian['links'].append(temp)
#     except:
# #         print searchword
#         count2 += 1
# #         print count2
# #         print a[1]
        



In [14]:

# print type((handian['nodes'][0]['weight']).item())
# for a in handian['nodes']:
#     a['weight'] = a['weight'].item()
for a in handian['links']:
    a['value'] = a['value'].item()


In [15]:
import numpy
import json
data = handian
with open(jsonpath, 'w') as f:
    f.write(json.dumps(data))

In [16]:
printlist(handian)

{'nodes': [{'color': '#4daf4a', 'depth': 0, 'name': 'T2', 'words': '年, 王, 帝, 太, 书, 宗, 元, 时, 文, 国'}, {'color': '#974fa2', 'depth': 0, 'name': 'T19', 'words': '能, 天, 臣, 民, 死, 行, 时, 道, 心, 已'}, {'color': '#e41a1c', 'depth': 0, 'name': 'T16', 'words': '卷, 花, 风, 山, 时, 春, 日, 归, 月, 年'}, {'color': '#fffe33', 'depth': 0, 'name': 'T13', 'words': '山, 金, 风, 日, 诗, 玉, 高, 生, 飞, 光'}, {'color': '#a7572a', 'depth': 0, 'name': 'T14', 'words': '军, 兵, 将, 州, 城, 遣, 战, 贼, 马, 守'}, {'color': '#4daf4a', 'depth': 0, 'name': 'T10', 'words': '心, 道, 生, 理, 学, 物, 能, 天, 处, 明'}, {'color': '#a7572a', 'depth': 0, 'name': 'T6', 'words': '官, 本, 钱, 年, 臣, 日, 百, 路, 州, 司'}, {'color': '#397db6', 'depth': 0, 'name': 'T9', 'words': '德, 阙, 天, 圣, 臣, 心, 道, 将, 神, 明'}, {'color': '#974fa2', 'depth': 0, 'name': 'T11', 'words': '天, 乐, 地, 物, 音, 声, 象, 正, 分, 神'}, {'color': '#fffe33', 'depth': 0, 'name': 'T5', 'words': '气, 服, 热, 水, 病, 黄, 治, 血, 寒, 阳'}, {'color': '#397db6', 'depth': 0, 'name': 'T1', 'words': '卷, 书, 文, 本, 诗, 传, 集, 记, 经, 纪'}, {'co